## Preprocessing

In [12]:
import pandas as pd
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import json
import pandas
import os
import io
import random
import json
from nltk.corpus import stopwords
import string
import numpy as np
from scipy import stats
from gsitk.preprocess import pprocess_twitter

Read file

In [14]:
# with this file we take the tweets from the database of hate_labeled_data.csv and put in a file, with each line a tweet
data = pd.read_csv('hate_labeled_data.csv')
data = data.rename(columns={'count': 'counter', 'class': 'type'})   # change names so they dont interfere with class names
data = data.sort_values(by='counter', ascending=False)

In [336]:
data.drop(columns='Unnamed: 0')

,counter,hate_speech,offensive_language,neither,type,tweet
14007,9,1,8,0,1,rt <allcaps> <user>: <user> <user> nah boa tha...
7415,9,2,7,0,1,a bitch like you need to get fucked right ever...
13370,9,2,7,0,1,nikko a gay bitch
11155,9,0,3,6,2,i'd rather follow some girls on instagram rath...
20709,9,1,7,1,1,she a hoe cause your boyfriend want her? she s...
...,...,...,...,...,...,...
8516,3,0,3,0,1,"confident in one way, pussy in another. so str..."
8514,3,0,3,0,1,"completely off topic, idgaf about that pretty ..."
8513,3,0,3,0,1,"commentators bitch at bird for ""sliding"" but <..."
8512,3,0,3,0,1,comin from a fat bitch


Preprocess the twits with gsitk tool

In [15]:
data['tweet'] = data['tweet'].map(pprocess_twitter.preprocess)

In [16]:
# divide the dataset into the category it belongs
# we are going to make two tests, one with offensive language and one with hate_speech

# separate datasets into neither, offensive and hate speech
data_hate = data[data.type == 0][['counter', 'type', 'tweet']]
data_offensive = data[data.type == 1][['counter', 'type', 'tweet']]
data_neither = data[data.type == 2][['counter', 'type', 'tweet']]

print('Hate twits', data_hate.shape)    # ->1430
print('Offensive twits', data_offensive.shape)  # -> 19190
print('Neither twits', data_neither.shape)  # -> 4163



Hate twits (1430, 3)
Offensive twits (19190, 3)
Neither twits (4163, 3)


using stratify to use all the information and maintain the proportion

In [23]:
dataset_hate = data_hate.append(data_neither)
dataset_hate_label = dataset_hate['type']
train_hate, test_hate, train_hate_label, test_hate_label = train_test_split(
    dataset_hate['tweet'], dataset_hate_label, test_size = 0.2, shuffle = True, stratify=dataset_hate_label)

dataset_off = data_offensive.append(data_neither)
dataset_off_label = dataset_off['type']
train_off, test_off, train_off_label, test_off_label = train_test_split(dataset_off['tweet'], dataset_off_label,
                                                                      test_size = 0.2, shuffle=True, stratify=dataset_off_label)



In [43]:

test_hate = test_hate.to_frame()
test_hate['label'] = test_hate_label
train_hate.keys()

test_off = test_off.to_frame()
test_off['label'] = test_off_label

In [44]:
test_off

,tweet,label
390,"""<user>: &<hastag> <number>;<user>: y'all thin...",1
7146,<user> i'd give that hoe the finger everyday a...,1
8650,dat bitch gotta lay on her back to pee <url>,1
9846,hoes always blaming hoes for being hoes.,1
15912,rt <allcaps> <user>: my bitch better lie to me...,1
...,...,...
15004,rt <allcaps> <user> if k michelle was apart of...,1
16700,rt <allcaps> <user>: &<hastag> <number>;<user>...,1
1058,&<hastag> <number>;&#<number>;&#<number>;&#<nu...,1
4579,<user> so says the one who fills an empty hole...,1


In [ ]:
# # as we have dispair number of tweets for each class, and we want the data to be balanced
# # we will make the two datasets with the minimum of the values as the limiting one
# # concatenate both dataframes into a final one
# dataset_offensive = data_offensive[0:5000].append(data_neither)
# dataset_hate = data_hate.append(data_neither[0:2000])
# # we shuffle it so the order is random and twits offensive and neither are mixed
# dataset_offensive = shuffle(dataset_offensive)
# dataset_hate = shuffle(dataset_hate)
# # divide into train/test sets
# train_offensive, test_offensive = train_test_split(dataset_offensive, test_size=0.2)
# train_hate, test_hate = train_test_split(dataset_hate, test_size=0.2)

Save train files as txt and test files as csv

In [45]:
# create or erase content of file in case it is exists
f = open('offensive_twits_train.txt', 'w+')
for twit in train_off['tweet']:
    f.write(twit + '\n')
f.close()

f = open('hate_twits_train.txt', 'w+')
for twit in train_hate['tweet']:
    f.write(twit + '\n')
f.close()

# save test files as csv
test_off.to_csv('offensive_twits_test.csv')
test_hate.to_csv('hate_twits_test.csv')

###### Execute 'hatedictionary.py hate_twits_train.txt hate' before continuing with the next part

## Evaluation

In [417]:
import json
import pandas
import os
import io
import random
import json
from nltk.corpus import stopwords
import string
import numpy as np
from scipy import stats
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

get the dictionaries

In [418]:
# hate
with open('dictionaries/hate_densify.json') as json_file:
    hate_dict_densify = json.load(json_file)
with open('dictionaries/hate_socialsent.json') as json_file:
    hate_dict_socialsent = json.load(json_file)
    
# offensive
with open('dictionaries/offensive_densify.json') as json_file:
    offensive_dict_densify = json.load(json_file)
with open('dictionaries/offensive_socialsent.json') as json_file:
    offensive_dict_socialsent = json.load(json_file)

In [419]:
len(hate_dict_socialsent)

15106

In [420]:
max(list(hate_dict_densify.values()))

4.314330577850342

In [421]:
sorted_keys_socialsent = sorted(hate_dict_socialsent, key=hate_dict_socialsent.get)
sorted_keys_densify= sorted(hate_dict_densify, key=hate_dict_densify.get)

In [466]:
print('Negative values: \n', sorted_keys_socialsent[0:50])
print('Positive values: \n', sorted_keys_socialsent[-50:])

Negative values: 
 ['nigger', 'motherfucker', 'faggot', 'asshole', 'fucker', 'cunt', 'dickhead', 'fag', 'bastard', 'skank', 'whore', 'douchebag', 'twat', 'chav', 'moron', 'jerk', 'dork', 'cocksucker', 'faggots', 'fucka', 'tramp', 'cunts', 'prick', 'licker', 'fags', 'mothafucka', 'wetback', 'loser', 'niggers', 'assholes', 'mudda', 'slut', 'queer', 'muthafucka', 'motha', 'turd', 'fuckers', 'twats', 'motherfuckers', 'bastards', 'dyke', 'mothafucker', 'dickheads', 'jerks', 'whores', 'dumbass', 'douche', 'scumbags', 'pussies', 'suckers']
Positive values: 
 ['enjoy', 'unreal', 'amazin', 'loved', 'epic', 'addition', 'fun', 'performance', 'impeccable', 'favorite', 'tremendous', 'talented', 'unbelievable', 'perfection', 'beauty', 'inspirational', 'nice', 'special', 'flawless', 'iconic', 'exceptional', 'atmosphere', 'magical', 'excellent', 'inspired', 'favourite', 'beautiful', 'gorgeous', 'inspiration', 'perfect', 'impressive', 'fab', 'exciting', 'inspiring', 'marvelous', 'great', 'best', 'remar

In [467]:
print('Negative values: \n', sorted_keys_densify[0:50])
print('Positive values: \n', sorted_keys_densify[-50:])

Negative values: 
 ['hoeass', 'faggit', 'fuckboy', 'wetback', 'bitchnigga', 'custy', 'biiiiiiiitch', 'spic', 'brolic', 'biiiiitch', 'wigger', 'dumass', 'shutcho', 'wicha', 'bitch-ass', 'greaseball', 'fucknigga', 'botch', 'brokeass', 'gtfoh', 'cockeyed', 'dyke', 'bitcccccch', 'stankin', 'muthafucka', 'gtf', 'bitchk', 'nigger', 'cornball', 'witta', 'forf', 'niggar', 'fym', 'bitchassness', 'prude', 'baldheaded', 'puzzy', 'worsum', 'darkie', 'innat', 'mothafucker', 'nigggga', 'stanking', 'getcho', 'beaner', 'fucktard', 'mandingo', 'baldhead', 'snitch', 'mfka']
Positive values: 
 ['latest', 'celebrate', 'performance', 'excited', 'enjoying', 'congratulations', 'day', 'inspired', 'sharing', 'appreciate', 'brazil', 'quality', 'nice', 'germany', 'exciting', 'good', 'present', 'inspiration', 'truly', 'blog', 'perfect', 'support', 'evening', 'wish', 'shared', 'favourite', 'thanks', 'hope', 'inspiring', 'lots', 'excellent', 'beautiful', 'wishes', 'loved', 'love', 'enjoyed', 'best', 'awesome', 'hap

In [424]:
sorted_keys_socialsent_off = sorted(offensive_dict_socialsent, key=offensive_dict_socialsent.get)
sorted_keys_densify_off = sorted(offensive_dict_densify, key=offensive_dict_densify.get)

In [468]:
print('Negative values: \n', sorted_keys_socialsent_off[0:50])
print('Positive values: \n', sorted_keys_socialsent_off[-50:])

Negative values: 
 ['nigga', 'motherfucker', 'asshole', 'skank', 'faggot', 'wetback', 'nigger', 'gringo', 'laat', 'chupacabra', 'redneck', 'dork', 'homie', 'puta', 'fucker', 'wiggy', 'bullshit', 'runt', 'goni', 'niggaz', 'jingo', 'mulatta', 'slut', 'shitty', 'denken', 'gooks', 'hoser', 'yowza', 'jigga', 'quisiera', 'crybaby', 'gurl', 'haha', 'onna', 'snitch', 'kaboom', 'hobo', 'goddamn', 'streetz', 'speedwagon', 'clich', 'no-brainer', 'drita', 'poppa', 'pasa', 'vivir', 'ohh', 'bitch', 'cunt', 'benzo']
Positive values: 
 ['skills', 'forever', 'performance', 'humble', 'neat', 'superb', 'quiet', 'exceptional', 'loves', 'impressive', 'perfectly', 'lucky', 'pleasure', 'passion', 'breezy', 'talent', 'cared', 'wonders', 'dreams', 'luck', 'beautifully', 'skill', 'grateful', 'elegant', 'proud', 'enjoy', 'enjoying', 'flawless', 'charming', 'lovers', 'dream', 'wonderful', 'brilliant', 'gorgeous', 'nice', 'touches', 'beauty', 'decent', 'beautiful', 'loving', 'fortunate', 'happy', 'good', 'delightf

In [469]:
print('Negative values: \n', sorted_keys_densify_off[0:50])
print('Positive values: \n', sorted_keys_densify_off[-50:])

Negative values: 
 ['yh', 'blac', 'grr', 'jacker', 'fpd', 'ey', 'hehe', 'cuh', 'rii', 'otf', 'kds', 'pasa', 'republicano', 'odia', 'drita', 'fml', 'bap', 'h-town', 'weet', 'samuela', 'ped', 'sav', 't-rex', 'sellin', 'nbp', 'chav', 't-bird', 'yardie', 'skank', 'pers', 'simp', 'joda', 'viri', 'shite', 'kik', 'mosa', 'wiggers', 'allisons', 'ily', 'chuckers', 'beaner', 'eww', 'lha', 'vap', 'gyp', 'mhm', 'pedo', 'uu', 'lls', 'boner']
Positive values: 
 ['spent', 'taking', 'always', 'though', 'none', 'seeing', 'home', 'room', 'seemed', 'giving', 'time', 'moment', 'beyond', 'staying', 'still', 'far', 'impressed', 'find', 'impressive', 'bringing', 'quite', 'excellent', 'perhaps', 'making', 'close', 'chance', 'brings', 'equally', 'way', 'much', 'rest', 'lot', 'place', 'enough', 'wonderful', 'interesting', 'looked', 'good', 'plenty', 'couple', 'yet', 'opportunity', 'moments', 'finding', 'looking', 'besides', 'comfortable', 'well', 'experience', 'shared']


In [461]:
sorted_keys_socialsent_off[:50]

['nigga',
 'motherfucker',
 'asshole',
 'skank',
 'faggot',
 'wetback',
 'nigger',
 'gringo',
 'laat',
 'chupacabra',
 'redneck',
 'dork',
 'homie',
 'puta',
 'fucker',
 'wiggy',
 'bullshit',
 'runt',
 'goni',
 'niggaz',
 'jingo',
 'mulatta',
 'slut',
 'shitty',
 'denken',
 'gooks',
 'hoser',
 'yowza',
 'jigga',
 'quisiera',
 'crybaby',
 'gurl',
 'haha',
 'onna',
 'snitch',
 'kaboom',
 'hobo',
 'goddamn',
 'streetz',
 'speedwagon',
 'clich',
 'no-brainer',
 'drita',
 'poppa',
 'pasa',
 'vivir',
 'ohh',
 'bitch',
 'cunt',
 'benzo']

Function to normalize the dictionary between -1 and 1 values

In [427]:
list(hate_dict_socialsent.keys())[np.argmin(list(hate_dict_socialsent.values()))]

'nigger'

In [428]:
# def normalize_dict(dictionary):

#     value_list = list(dictionary.values())
#     key_list = list(dictionary.keys())
#     max_list = max(value_list)
#     min_list = min(value_list)
#     a = {key_list[i]: 2 * (value_list[i] - min_list)/(max_list - min_list) -1  for i in range(len(key_list))}
#     return a

In [429]:
# # hate
# hate_dict_socialsent = normalize_dict(hate_dict_socialsent)
# hate_dict_densify = normalize_dict(hate_dict_densify)

# # offensive
# offensive_dict_socialsent = normalize_dict(offensive_dict_socialsent)
# offensive_dict_densify = normalize_dict(offensive_dict_densify)

Read each test file 

In [430]:
test_hate = pd.read_csv('hate_twits_test.csv')
test_off = pd.read_csv('offensive_twits_test.csv')
# test_hate = hate_twits.drop(columns='Unnamed: 0')
# test_off = offensive_twits.drop(columns='Unnamed: 0')

In [431]:
# initialize arrays for evaluation
# ground truth
hate_label = np.zeros(len(test_hate))
offensive_label = np.zeros(len(test_off))
# evaluated value
hate_eval = np.zeros(len(test_hate))
hate_offensive = np.zeros(len(test_off))

In [432]:
hate_dict_densify.get('the')

###### Pearson coefficient

In [433]:
def evaluate(data, args):
    evaluation_soc = 0  # socialsent
    evaluation_dens = 0   # densify
    counter = 0
    if args == 'hate':
        dict_soc = hate_dict_socialsent
        dict_dens = hate_dict_densify
    elif args == 'offensive':
        dict_soc = offensive_dict_socialsent
        dict_dens = offensive_dict_densify
    dict_vector_soc = {}
    dict_vector_dens = {}
    for word in data['tweet'].split():
        if dict_soc.get(word) is not None:
            evaluation_soc += dict_soc.get(word)
            evaluation_dens += dict_dens.get(word)
            counter += 1
            dict_vector_soc[word] = dict_soc.get(word)
            dict_vector_dens[word] = dict_dens.get(word)
    if counter == 0:
        counter = 1   # avoid division by 0
    data['vector_soc'] = dict_vector_soc
    data['vector_dens'] = dict_vector_dens
    data['evaluation_soc'] = 1 if evaluation_soc/counter >= 0.5 else -1
    data['evaluation_dens'] = 1 if evaluation_dens/counter > 0 else -1
    data['label_eval'] = 1 if data['label'] == 2 else -1   # 1 if type is neither, -1 if is hate/offensive
    return data

In [434]:
hate_twits_evaluated = test_hate.apply(evaluate, args=('hate',), axis=1)
offensive_twits_evaluated = test_off.apply(evaluate, args=('offensive',), axis=1)

In [435]:

hate_eval_socialsent = stats.pearsonr(hate_twits_evaluated['evaluation_soc'], hate_twits_evaluated['label_eval'])
hate_eval_densify = stats.pearsonr(hate_twits_evaluated['evaluation_dens'], hate_twits_evaluated['label_eval'])
offensive_eval_socialsent = stats.pearsonr(offensive_twits_evaluated['evaluation_soc'], offensive_twits_evaluated['label_eval'])
offensive_eval_densify = stats.pearsonr(offensive_twits_evaluated['evaluation_dens'], offensive_twits_evaluated['label_eval'])
print(hate_eval_socialsent)
print(hate_eval_densify)
print(offensive_eval_socialsent)
print(offensive_eval_densify)
print('report hate socialsent: ', classification_report(hate_twits_evaluated['evaluation_soc'], hate_twits_evaluated['label_eval']))
print('report hate densify: ', classification_report(hate_twits_evaluated['evaluation_dens'], hate_twits_evaluated['label_eval']))
print('report offensive socialsent: ', classification_report(offensive_twits_evaluated['evaluation_soc'], offensive_twits_evaluated['label_eval']))
print('report offensive densify: ', classification_report(offensive_twits_evaluated['evaluation_dens'], offensive_twits_evaluated['label_eval'])) 

(0.22574736673722118, 2.138790155084657e-14)
(0.23822591420434278, 6.656830529007784e-16)
(0.15456233605962005, 2.2900494174449788e-26)
(0.12622894280169195, 4.74134564950935e-18)
report hate socialsent:                precision    recall  f1-score   support

          -1       0.94      0.31      0.46       869
           1       0.28      0.93      0.43       250

    accuracy                           0.45      1119
   macro avg       0.61      0.62      0.45      1119
weighted avg       0.79      0.45      0.46      1119

report hate densify:                precision    recall  f1-score   support

          -1       0.25      0.54      0.34       134
           1       0.93      0.78      0.85       985

    accuracy                           0.75      1119
   macro avg       0.59      0.66      0.60      1119
weighted avg       0.84      0.75      0.79      1119

report offensive socialsent:                precision    recall  f1-score   support

          -1       0.22      0.94 

In [490]:
print(hate_twits_evaluated[0:1]['vector_soc'][0])

{'made': 0.4949378937768095, 'greedy': 0.10941989327217277, 'faggot': 0.021329974704203693, 'atleast': 0.3638986162080378, 'broner': 0.48111176127377486}


###### Logistic regression

First, create a DictVectorizer object and create a vector with all the features( each word is a feature).
Each twit is transformed into a dictionary with word:value, and then with all the dictionaries of all the train and test twits is created the vector mapping.

In [436]:
# join train and test into a dataset to make the dictionary for every instance
hate_twits_all = train_hate.append(test_hate)
off_twits_all = train_off.append(test_off)

In [437]:
print(len(train_hate))
print(len(train_off))
hate_twits_all[len(train_hate):]

4474
18682


,tweet,label,Unnamed: 0
0,<user> yea. <repeat>what did it do for maidana? it made him a greedy faggot loser. atleast broner is black...,0,2636.0
1,lmao <elong> @ supreme being trash. guess bape is trash to huh?,2,12538.0
2,rt <allcaps> <user>: derek jeter leaves baseball as one of the greatest yankees ever. (via <user>) <url>,2,17396.0
3,we over here struggling with college and work and ice jj fish making money for being trash lol. what has t...,2,22533.0
4,"""the devil grows inside the hearts of the selfish and wicked\nwhite, brown, yellow and black color is not ...",2,561.0
...,...,...,...
1114,you said you be smoking strong but your wrong you been smoking trash it's not in my lungs if it ain't that...,2,23404.0
1115,rt <allcaps> <user>: every seth mcfarlene cartoon is trash yall love low iq comedy,2,16711.0
1116,<user> wha? me? i'm just waiting for showtime to bring back queer as folk.,2,7191.0
1117,"yep, sources confirmed it. that iggy concert last night was trash",2,23154.0


In [438]:
def create_dict_value(data, args):
    counter = 0
    if args == 'hate':
        dict_soc = hate_dict_socialsent
        dict_dens = hate_dict_densify
    elif args == 'offensive':
        dict_soc = offensive_dict_socialsent
        dict_dens = offensive_dict_densify
    dict_vector_soc = {}   # socialsent
    dict_vector_dens = {}  # densify
    for word in data['tweet'].split():
        if dict_soc.get(word) is not None:
            dict_vector_soc[word] = dict_soc.get(word)
            dict_vector_dens[word] = dict_dens.get(word)
    data['vector_soc'] = dict_vector_soc
    data['vector_dens'] = dict_vector_dens
    data['label_eval'] = 1 if data['label'] == 2 else 0   # 1 if type is neither, -1 if is hate/offensive
    return data

In [439]:
hate_twits_all = hate_twits_all.apply(create_dict_value, args=('hate',), axis=1)
off_twits_all = off_twits_all.apply(create_dict_value, args=('offensive',), axis=1)

In [440]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_colwidth', 110)
display(hate_twits_all['vector_soc'])
print('shape: ', hate_twits_all['vector_soc'].shape)

20047    {'rt': 0.5063806141046642, 'back': 0.5263003434024257, 'cause': 0.2677402089969097, 'see': 0.5334494047145...
7465     {'random': 0.40529974965132687, 'scroll': 0.49962803387642474, 'charlie': 0.4868830320709538, 'horses': 0....
21686    {'side': 0.4315926665743458, 'trip': 0.7697714431763033, 'dc': 0.49483151480534737, 'charlie': 0.486883032...
9196     {'fresh': 0.6160448856578564, 'gym': 0.5376617430152593, 'yet': 0.5258872116549106, 'bout': 0.121558781753...
7841     {'pussy': 0.09616483947559788, 'lounge': 0.617986187862427, 'killin': 0.15160235388139465, 'girlz': 0.1808...
                                                             ...                                                      
1114     {'said': 0.47912170683458416, 'smoking': 0.30964937457548797, 'strong': 0.6116551535552796, 'wrong': 0.490...
1115     {'rt': 0.5063806141046642, 'every': 0.549755445910003, 'seth': 0.4904054306915105, 'cartoon': 0.5449714493...
1116     {'waiting': 0.5523911135733949, 'showti

shape:  (5593,)


Create the vector

In [441]:
v = DictVectorizer(sparse=True)
v_hate_soc = v.fit_transform(hate_twits_all['vector_soc'])
v_hate_dens = v.fit_transform(hate_twits_all['vector_dens'])

v_off_soc = v.fit_transform(off_twits_all['vector_soc'])
v_off_dens = v.fit_transform(off_twits_all['vector_dens'])

In [442]:
print(v_hate_soc.shape)
print(v_hate_dens.shape)
print(v_off_soc.shape)
print(v_off_dens.shape)

(5593, 7522)
(5593, 7522)
(23353, 12111)
(23353, 12111)


In [443]:
np.set_printoptions(edgeitems=8)
print(v_hate_soc)
print('shape; ', v_hate_soc.shape)

  (0, 428)	0.5263003434024257
  (0, 793)	0.3164999213872843
  (0, 1064)	0.2677402089969097
  (0, 3885)	0.47165527548585323
  (0, 5578)	0.5063806141046642
  (0, 5758)	0.5334494047145935
  (0, 6823)	0.21597066359161696
  (1, 1124)	0.4868830320709538
  (1, 1689)	0.8236881043634942
  (1, 1987)	0.38211414617804745
  (1, 2155)	0.5014826966725354
  (1, 3134)	0.3535550463720502
  (1, 4071)	0.4747581413735531
  (1, 5259)	0.40529974965132687
  (1, 5733)	0.49962803387642474
  (1, 5815)	0.21251174491134672
  (1, 5938)	0.5873525549082252
  (1, 7205)	0.3818910016657658
  (2, 1124)	0.4868830320709538
  (2, 1521)	0.30862250023755056
  (2, 1694)	0.49483151480534737
  (2, 4705)	0.3183618464339526
  (2, 5922)	0.4315926665743458
  (2, 6860)	0.7697714431763033
  (3, 780)	0.12155878175398618
  :	:
  (5589, 3907)	0.504886118759416
  (5589, 5578)	0.5063806141046642
  (5589, 5805)	0.4904054306915105
  (5589, 6823)	0.21597066359161696
  (5589, 7447)	0.09671093351529594
  (5590, 428)	0.5263003434024257
  (5590, 

Fit a logisitic classifier with the training data only

In [444]:
print('Train: ', v_hate_soc[:len(train_hate)].shape)
print('Test: ', v_hate_soc[len(train_hate):].shape)
print('Total: ', v_hate_soc.shape)


Train:  (4474, 7522)
Test:  (1119, 7522)
Total:  (5593, 7522)


In [445]:
clf_hate_soc = LogisticRegression( random_state=0).fit(v_hate_soc[:len(train_hate)], hate_twits_all['label_eval'][:len(train_hate)])
clf_hate_dens = LogisticRegression( random_state=0).fit(v_hate_dens[:len(train_hate)], hate_twits_all['label_eval'][:len(train_hate)])
clf_off_soc = LogisticRegression(max_iter = 5000, random_state=0).fit(v_off_soc[:len(train_off)], off_twits_all['label_eval'][:len(train_off)])
clf_off_dens = LogisticRegression(max_iter = 5000, random_state=0).fit(v_off_dens[:len(train_off)], off_twits_all['label_eval'][:len(train_off)])


Predict class of test data

In [446]:
y_score_hate_soc = clf_hate_soc.predict(v_hate_soc[len(train_hate):])
y_score_hate_dens = clf_hate_dens.predict(v_hate_dens[len(train_hate):])
y_score_off_soc = clf_off_soc.predict(v_off_soc[len(train_off):])
y_score_off_dens = clf_off_dens.predict(v_off_dens[len(train_off):])

In [447]:
precision_hate_soc = average_precision_score(hate_twits_all['label_eval'][len(train_hate):], y_score_hate_soc)
precision_hate_dens = average_precision_score(hate_twits_all['label_eval'][len(train_hate):], y_score_hate_dens)
precision_off_soc = average_precision_score(off_twits_all['label_eval'][len(train_off):], y_score_off_soc)
precision_off_dens = average_precision_score(off_twits_all['label_eval'][len(train_off):], y_score_off_dens)

In [448]:
print('Precision hate socialsent: ', precision_hate_soc)
print('Precision hate densify: ',precision_hate_dens)
print('Precision offensive socialsent: ',precision_off_soc)
print('Precision offensive densify: ',precision_off_dens)

Precision hate socialsent:  0.7978482906232198
Precision hate densify:  0.8983355298745312
Precision offensive socialsent:  0.5741220443179291
Precision offensive densify:  0.5976600189309137


In [459]:
print(stats.pearsonr(hate_twits_all['label_eval'][len(train_hate):], y_score_hate_soc))
print(stats.pearsonr(hate_twits_all['label_eval'][len(train_hate):], y_score_hate_dens))
print(stats.pearsonr(off_twits_all['label_eval'][len(train_off):], y_score_off_soc))
print(stats.pearsonr(off_twits_all['label_eval'][len(train_off):], y_score_off_dens))
print('Report hate soc: ', classification_report(hate_twits_all['label_eval'][len(train_hate):], y_score_hate_soc))
print('Report hate dens: ',classification_report(hate_twits_all['label_eval'][len(train_hate):], y_score_hate_dens))
print('Report off soc: ',classification_report(off_twits_all['label_eval'][len(train_off):], y_score_off_soc))
print('Report off dens: ',classification_report(off_twits_all['label_eval'][len(train_off):], y_score_off_dens))



(0.40956153269530765, 1.690701798089343e-46)
(0.7283164897672423, 1.4193624106992377e-185)
(0.6590951695142439, 0.0)
(0.6835435775264599, 0.0)
Report hate soc:                precision    recall  f1-score   support

           0       0.84      0.28      0.42       286
           1       0.80      0.98      0.88       833

    accuracy                           0.80      1119
   macro avg       0.82      0.63      0.65      1119
weighted avg       0.81      0.80      0.76      1119

Report hate dens:                precision    recall  f1-score   support

           0       0.89      0.70      0.78       286
           1       0.90      0.97      0.94       833

    accuracy                           0.90      1119
   macro avg       0.90      0.83      0.86      1119
weighted avg       0.90      0.90      0.90      1119

Report off soc:                precision    recall  f1-score   support

           0       0.92      0.98      0.95      3838
           1       0.85      0.59      0

In [449]:
hate_twits_all['label_eval'][len(train_hate):].describe()


count    1119.000000
mean        0.744415
std         0.436385
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: label_eval, dtype: float64

###### Classification on unigram

Create a matrix of the ocurrences each word occurs in a twit

In [450]:
vectorizer = CountVectorizer()
unigram_ocurrence_matrix = vectorizer.fit_transform(hate_twits_all['tweet'])
unigram_ocurrence_matrix_off = vectorizer.fit_transform(off_twits_all['tweet'])
#print(vectorizer.get_feature_names())
print(unigram_ocurrence_matrix_off.toarray()[0].shape)

(19102,)


Fit a logistic regresor with this features

In [451]:
clf_unigram = LogisticRegression(random_state=0).fit(unigram_ocurrence_matrix[:len(train_hate)], hate_twits_all['label_eval'][:len(train_hate)])
clf_unigram_off = LogisticRegression(random_state=0, max_iter=5000).fit(unigram_ocurrence_matrix_off[:len(train_off)], off_twits_all['label_eval'][:len(train_off)])


In [452]:
y_score_unigram = clf_unigram.predict(unigram_ocurrence_matrix[len(train_hate):])
y_score_unigram_off = clf_unigram_off.predict(unigram_ocurrence_matrix_off[len(train_off):])

In [460]:
precision_unigram = average_precision_score(hate_twits_all['label_eval'][len(train_hate):], y_score_unigram)
precision_unigram_off = average_precision_score(off_twits_all['label_eval'][len(train_off):], y_score_unigram_off)
precision_unigram
print(stats.pearsonr(hate_twits_all['label_eval'][len(train_hate):], y_score_unigram))
print(stats.pearsonr(off_twits_all['label_eval'][len(train_off):], y_score_unigram_off))
print(classification_report(hate_twits_all['label_eval'][len(train_hate):], y_score_unigram))
print(classification_report(off_twits_all['label_eval'][len(train_off):], y_score_unigram_off))

(0.8046536737951281, 3.8256357708205807e-255)
(0.8591798580421246, 0.0)
              precision    recall  f1-score   support

           0       0.91      0.79      0.85       286
           1       0.93      0.97      0.95       833

    accuracy                           0.93      1119
   macro avg       0.92      0.88      0.90      1119
weighted avg       0.93      0.93      0.93      1119

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      3838
           1       0.86      0.91      0.88       833

    accuracy                           0.96      4671
   macro avg       0.92      0.94      0.93      4671
weighted avg       0.96      0.96      0.96      4671



In [454]:
X[0]

<1x10787 sparse matrix of type '<class 'numpy.int64'>'
	with 18 stored elements in Compressed Sparse Row format>

In [455]:
hate_twits_all['tweet'][:len(train_hate)]

20047       rt <allcaps> <user>: i'm only back cause i don't wanna see the boys lose to trash <hastag> vote<number>sos
7465     a random am scroll lands me here. <repeat>rt <allcaps> <user> charlie horses durin sex just simply means y...
21686    there should be a side trip into dc <allcaps> to charlie palmers during cpac <allcaps> <user> <user> <user...
9196                                                             fresh out the gym but yet i'm bout to eat free birds.
7841                                                     at pussy cat lounge killin it with my girlz <user> and <user>
                                                             ...                                                      
5068     <user> hey i'll leave abbey too! oh if i get a job on the slope i might move back! cause id lose out on lo...
3330                                <user> <user> bc <user> is trash why would he lie about that? i told you he's real
2500                                            

In [456]:
hate_twits_all['label_eval'][len(train_hate):][hate_twits_all['label_eval'][len(train_hate):] == 0]

0       0
6       0
12      0
14      0
17      0
       ..
1093    0
1097    0
1100    0
1112    0
1113    0
Name: label_eval, Length: 286, dtype: int64

In [457]:
hate_dict_socialsent.get('think')

0.49625832763748035